# DISTIL: Backdoor Detection Pipeline for Object Detection Models


## Introduction

This notebook implements a detection pipeline for identifying potential backdoors in object detection models (SSD and Faster R-CNN). The pipeline generates triggers that can potentially manipulate model predictions, helping to identify potential backdoors.


## Setup and Imports


In [ ]:
import os
import sys
sys.path.append('../../../')

In [ ]:
!pip install git+https://github.com/openai/glide-text2im

In [ ]:
!git clone https://github.com/ex3ploiter/DISTIL.git

In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from typing import List, Tuple, Dict, Optional, Union, Any
from tqdm.auto import tqdm
from math import ceil
import glob
import tarfile


# Update imports to use existing project structure
import sys

from DISTIL.DISTIL.utils.metrics import get_model_state
from DISTIL.DISTIL.models.loader import load_detection_model, get_classifier_layer
from DISTIL.DISTIL.models.similarity import greedy_class_farthest
from DISTIL.DISTIL.utils.transforms import transform_image, transform_image_tensor
from DISTIL.DISTIL.detection.trigger_generator import (
    SSDTriggerGenerator,
    FRCNNTriggerGenerator,
    trigger_evaluation
)
from DISTIL.DISTIL.evaluation.auc import load_results_and_compute_auc
from DISTIL.DISTIL.detection import backdoor_detector_objDetection

## Configuration


In [ ]:
# Global configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEFAULT_ROOT_DIR = "./object-detection-jul2022-train/models"
DEFAULT_OUTPUT_DIR = "./outputs"
DEFAULT_ARCHITECTURE = "ssd"  
DEFAULT_OUTPUT_FILE = os.path.join(DEFAULT_OUTPUT_DIR, f"detection_results_{DEFAULT_ARCHITECTURE}.txt")
DEFAULT_GUIDANCE_SCALE = 25
DEFAULT_TIMESTEPS = 50
TRIGGER_CONFIDENCE_THRESHOLD = 0.95

In [18]:
# Environment variables can override defaults
ARCHITECTURE = os.getenv("TROJDET_ARCHITECTURE", DEFAULT_ARCHITECTURE)
ROOT_DIR = os.getenv("TROJDET_ROOT_DIR", DEFAULT_ROOT_DIR)
OUTPUT_DIR = os.getenv("TROJDET_OUTPUT_DIR", DEFAULT_OUTPUT_DIR)
OUTPUT_FILE = os.getenv("TROJDET_OUTPUT_FILE", DEFAULT_OUTPUT_FILE)

SummaryFiles = list()
summary_file = os.path.join(OUTPUT_DIR, f"overall_trigger_summary_{ARCHITECTURE}.txt")
SummaryFiles.append(summary_file)

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Using device: {DEVICE}")
print(f"Architecture: {ARCHITECTURE}")
print(f"Root directory: {ROOT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Summary File: {summary_file}")

Using device: cuda
Architecture: ssd
Root directory: /teamspace/studios/this_studio/object-detection-jul2022-train/models
Output directory: ./outputs
Summary File: ./outputs/overall_trigger_summary_ssd.txt


## Prepare Models

In [ ]:
#!gdown --folder 11khneOUXZPSMHPPObNSyl1ObLsO6ohoV -O 'object-detection-jul2022-train'

In [ ]:
def extract_tar_gz_files_in_folder(folder_path):
    # Loop through all files in the specified folder
    for item in os.listdir(folder_path):
        # Check if the item is a tar.gz file
        if item.endswith('.tar.gz'):
            tar_file_path = os.path.join(folder_path, item)
            # Create a directory to extract the contents
            extract_folder = os.path.join(folder_path)
            os.makedirs(extract_folder, exist_ok=True)  # Create the directory if it doesn't exist

            # Extract the tar.gz file
            with tarfile.open(tar_file_path, 'r:gz') as tar_ref:
                tar_ref.extractall(extract_folder)
                print(f'Extracted {item} to {extract_folder}')

extract_tar_gz_files_in_folder(DEFAULT_ROOT_DIR)


## Define the model folder processing function

In [22]:
def process_model_folder(
    model_path: str,
    output_file: str = OUTPUT_FILE
) -> Optional[Tuple[str, str, List[float], float]]:
    """
    Process a single model folder to detect potential backdoors.
    
    Args:
        model_path: Path to the model folder
        output_file: File to write detailed results
        
    Returns:
        Model results summary or None if skipped
    """
    folder_name = os.path.basename(model_path)
    ckpt_file = os.path.join(model_path, "model.pt")
    
    if not os.path.isfile(ckpt_file):
        # print(f"[SKIP] No model.pt in {model_path}.")
        return None
        
    # print(f"\n=== Attempting to load detection model from '{ckpt_file}' ===")
    detection_model = load_detection_model(ckpt_file, architecture=ARCHITECTURE)
    if detection_model is None:
        # print(f"[SKIP] Checkpoint '{ckpt_file}' is not recognized as {ARCHITECTURE}. Moving on.")
        return None
    # print("  -> Detection model loaded successfully.")
    
    # Get classifier head for detection model
    classifier_head = get_classifier_layer(detection_model)
    # print("  -> Computing farthest labels with 'greedy_class_farthest'...")
    far_list = greedy_class_farthest(detection_model)
    far_map = {i: j for (i, j, sim_val) in far_list}
    
    
    model_state = get_model_state(model_path)
    # print(f"  -> Model state: {model_state}")
    
    # Create output directory for triggers
    triggers_out_dir = os.path.join(
        os.path.join(OUTPUT_DIR, folder_name),
        "generated_triggers"
    )
    os.makedirs(triggers_out_dir, exist_ok=True)
    
    # Check for clean example data
    clean_data_dir = os.path.join(model_path, "clean-example-data")
    if not os.path.isdir(clean_data_dir):
        # print(f"[SKIP] No 'clean-example-data' in {model_path}.")
        return None
        
    # Get clean image files
    image_files = [
        os.path.join(clean_data_dir, fname)
        for fname in sorted(os.listdir(clean_data_dir))
        if fname.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    if not image_files:
        # print(f"[SKIP] No clean example images found in '{clean_data_dir}'.")
        return None
        
    model_score_list = []
    results_list = []
    
    # Process a subset of images (adjust based on architecture)
    image_limit = 1 if ARCHITECTURE.lower() == 'frcnn' else 4
    
    for sample_image_path in image_files[:image_limit]:
        # print(f"  -> Processing image: '{sample_image_path}'")
        cv_image = cv2.imread(sample_image_path)
        if cv_image is None:
            # print(f"[SKIP] Cannot load '{sample_image_path}' via OpenCV.")
            continue
        cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        
        # Preprocess image
        t224 = transform_image_tensor(cv_image)  # Expected shape: [1, 3, 224, 224]
        pil_224 = transforms.ToPILImage()(t224.squeeze(0))
        pil_300 = transforms.Resize((300, 300))(pil_224)
        in_300 = transforms.ToTensor()(pil_300).unsqueeze(0).to(DEVICE)
        images = in_300
        
        # Run detection on preprocessed image
        with torch.no_grad():
            if ARCHITECTURE.lower() == 'frcnn' and hasattr(detection_model.model, "roi_heads"):
                detections = detection_model([in_300.squeeze(0)])
            else:
                detections = detection_model(in_300)
        
        detection = detections[0]
        boxes = detection["boxes"].cpu().numpy()
        scores = detection["scores"].cpu().numpy()
        labels = detection["labels"].cpu().numpy()
        
        # Set confidence threshold based on architecture
        confidence_threshold = 0.99 if ARCHITECTURE.lower() == 'frcnn' else 0.5
        # print(f"    -> Found {len(boxes)} boxes. Considering those with conf > {confidence_threshold}.")
        
        base_tensor = transforms.ToTensor()(pil_300).to(DEVICE)
        
        # Process each detected box
        for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
            result = backdoor_detector_objDetection.process_detection_box(
                box_index=i, 
                box_coords=box.astype(int),
                confidence=score,
                label=label,
                classifier_head=classifier_head,
                label_mapping=far_map,
                base_tensor=base_tensor,
                images=images,
                detection_model=detection_model,
                triggers_dir=triggers_out_dir,
                box_confidence_threshold=confidence_threshold
            )
            
            if result is not None:
                # Unpack the result properly, now including trigger_score
                box_index, coords, src_label, tgt_label, detection_confidence, trigger_conf, trigger_path, trigger_score = result
                results_list.append(result)
                # Store the trigger_score instead of trigger_conf
                model_score_list.append(trigger_score)
                
    # Write per-model results if any triggers were generated
    if model_score_list:
        avg_score = sum(model_score_list) / len(model_score_list)
        with open(output_file, "a") as f:
            f.write(f"\nModel folder: {folder_name}\n")
            f.write(f"  Model state: {model_state}\n")
            f.write(f"  Trigger scores: {model_score_list}\n")
            f.write(f"  Average trigger score: {avg_score}\n")
            for result in results_list:
                idx, coords, src, tgt, det_conf, trig_conf, trig_file, trig_score = result
                f.write(f"  Box#{idx} coords={coords}, label: {src}->{tgt}, detection_conf={det_conf:.2f}, "
                        f"trigger_conf={trig_conf:.4f}, trigger_score={trig_score:.4f}, trigger='{trig_file}'\n")
            f.write("\n")
        return (folder_name, model_state, model_score_list, avg_score)
    else:
        # print(f"  -> No bounding boxes >{confidence_threshold} or triggers generated in this model.")
        return None


## Main detection pipeline function

In [21]:
def main_detection_trigger_pipeline(summary_file) -> None:
    """
    Main pipeline for detecting backdoors in object detection models.
    
    This function iterates through model folders in the ROOT_DIR,
    processes each model to find potential backdoors, and generates
    a detailed report of the findings.
    """
    os.environ['TROJDET_ARCHITECTURE']= ARCHITECTURE
    if not os.path.isdir(ROOT_DIR):
        print(f"[ERROR] '{ROOT_DIR}' doesn't exist.")
        return

    overall_results = []

    # Process each model folder
    for folder_name in tqdm(sorted(os.listdir(ROOT_DIR))):
        model_path = os.path.join(ROOT_DIR, folder_name)
        if not os.path.isdir(model_path):
            continue
            
        result = process_model_folder(model_path, OUTPUT_FILE)
        if result is not None:
            overall_results.append(result)

    # Generate summary report
    # print("\nFinished main_detection_trigger_pipeline!")
    with open(summary_file, "w") as sf:
        sf.write("Model\tState\tAvg_Trigger_Score\tScores\n")
        for folder_name, state, scores_, avg in overall_results:
            sf.write(f"{folder_name}\t{state}\t{avg}\t{scores_}\n")
            

In [23]:
main_detection_trigger_pipeline(summary_file)

  0%|          | 0/144 [00:00<?, ?it/s]

 [SKIP] '/teamspace/studios/this_studio/object-detection-jul2022-train/models/id-00000000/model.pt' references FasterRCNN, not SSD.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Global configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEFAULT_ROOT_DIR = "./object-detection-jul2022-train/models"
DEFAULT_OUTPUT_DIR = "./outputs"
DEFAULT_ARCHITECTURE = "frcnn" 
DEFAULT_OUTPUT_FILE = os.path.join(DEFAULT_OUTPUT_DIR, f"detection_results_{DEFAULT_ARCHITECTURE}.txt")
DEFAULT_GUIDANCE_SCALE = 25
DEFAULT_TIMESTEPS = 50
TRIGGER_CONFIDENCE_THRESHOLD = 0.95

In [11]:
# Environment variables can override defaults
os.environ["TROJDET_ARCHITECTURE"] = DEFAULT_ARCHITECTURE
ARCHITECTURE = os.getenv("TROJDET_ARCHITECTURE", DEFAULT_ARCHITECTURE)
ROOT_DIR = os.getenv("TROJDET_ROOT_DIR", DEFAULT_ROOT_DIR)
OUTPUT_DIR = os.getenv("TROJDET_OUTPUT_DIR", DEFAULT_OUTPUT_DIR)
OUTPUT_FILE = os.getenv("TROJDET_OUTPUT_FILE", DEFAULT_OUTPUT_FILE)

summary_file = os.path.join(OUTPUT_DIR, f"overall_trigger_summary_{ARCHITECTURE}.txt")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Using device: {DEVICE}")
print(f"Architecture: {ARCHITECTURE}")
print(f"Root directory: {ROOT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Summary File: {summary_file}")

Using device: cuda
Architecture: frcnn
Root directory: /teamspace/studios/this_studio/object-detection-jul2022-train/models
Output directory: ./outputs
Summary File: ./outputs/overall_trigger_summary_frcnn.txt


In [12]:
main_detection_trigger_pipeline(summary_file)
SummaryFiles.append(summary_file)

  0%|          | 0/144 [00:00<?, ?it/s]

 [SKIP] Could not unpickle '/teamspace/studios/this_studio/object-detection-jul2022-train/models/id-00000001/model.pt' as FasterRCNN: Can't get attribute 'SSD' on <module 'models'>
 [SKIP] Could not unpickle '/teamspace/studios/this_studio/object-detection-jul2022-train/models/id-00000002/model.pt' as FasterRCNN: Can't get attribute 'SSD' on <module 'models'>
 [SKIP] Could not unpickle '/teamspace/studios/this_studio/object-detection-jul2022-train/models/id-00000003/model.pt' as FasterRCNN: Can't get attribute 'SSD' on <module 'models'>
 [SKIP] Could not unpickle '/teamspace/studios/this_studio/object-detection-jul2022-train/models/id-00000005/model.pt' as FasterRCNN: Can't get attribute 'SSD' on <module 'models'>


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 [SKIP] Could not unpickle '/teamspace/studios/this_studio/object-detection-jul2022-train/models/id-00000143/model.pt' as FasterRCNN: Can't get attribute 'SSD' on <module 'models'>


In [27]:
load_results_and_compute_auc(SummaryFiles)


Final AUC‑ROC: 0.6803


0.6803141711229947